In [29]:
import torch
#!pip install -q torch-scatter~=2.1.0 torch-sparse~=0.6.16 torch-cluster~=1.6.0 torch-spline-conv~=1.2.1 torch-geometric==2.2.0 -f https://data.pyg.org/whl/torch-{torch.__version__}.html

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Graph Autoencoder (VAE) and Variational Graph Autoencoder (VGAE)


In [30]:
import numpy as np
np.random.seed(0)
import torch
torch.manual_seed(0)
import matplotlib.pyplot as plt
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True, add_negative_train_samples=False),
])

dataset = Planetoid('.', name='PubMed', transform=transform)

train_data, val_data, test_data = dataset[0]


In [32]:
print(f"Data set[0]:\n {dataset[0]}")
print(f"Train data: \n{train_data}")
print(f"Val data: \n{val_data}")
print(f"Test data: \n{test_data}")



Data set[0]:
 (Data(x=[19717, 500], edge_index=[2, 75352], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], pos_edge_label=[37676], pos_edge_label_index=[2, 37676]), Data(x=[19717, 500], edge_index=[2, 75352], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], pos_edge_label=[2216], pos_edge_label_index=[2, 2216], neg_edge_label=[2216], neg_edge_label_index=[2, 2216]), Data(x=[19717, 500], edge_index=[2, 79784], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], pos_edge_label=[4432], pos_edge_label_index=[2, 4432], neg_edge_label=[4432], neg_edge_label_index=[2, 4432]))
Train data: 
Data(x=[19717, 500], edge_index=[2, 75352], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], pos_edge_label=[37676], pos_edge_label_index=[2, 37676])
Val data: 
Data(x=[19717, 500], edge_index=[2, 75352], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], pos_edge_label=[2216], pos_edge_label_index=[2, 2216], neg_edge_

In [33]:
from torch_geometric.nn import GCNConv, VGAE


In [34]:
class Encoder(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.conv1 = GCNConv(dim_in, 2 * dim_out)
        self.conv_mu = GCNConv(2 * dim_out, dim_out)
        self.conv_logstd = GCNConv(2 * dim_out, dim_out)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [35]:
model = VGAE(Encoder(dataset.num_features, 16)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    loss = model.recon_loss(z, train_data.pos_edge_label_index) + (1 / train_data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    return model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)

for epoch in range(101):
    loss = train()
    val_auc, val_ap = test(test_data)
    if epoch % 10 == 0:
        print(f'Epoch {epoch:>2} | Loss: {loss:.4f} | Val AUC: {val_auc:.4f} | Val AP: {val_ap:.4f}') 

test_auc, test_ap = test(test_data) 
print(f'Test AUC: {test_auc:.4f} | Test AP {test_ap:.4f}')

Epoch  0 | Loss: 3.4529 | Val AUC: 0.8899 | Val AP: 0.8688
Epoch 10 | Loss: 1.7083 | Val AUC: 0.8908 | Val AP: 0.8635
Epoch 20 | Loss: 1.3152 | Val AUC: 0.8952 | Val AP: 0.8697
Epoch 30 | Loss: 1.2523 | Val AUC: 0.8940 | Val AP: 0.8731
Epoch 40 | Loss: 1.2273 | Val AUC: 0.8890 | Val AP: 0.8724
Epoch 50 | Loss: 1.2018 | Val AUC: 0.8992 | Val AP: 0.8838
Epoch 60 | Loss: 1.1425 | Val AUC: 0.9129 | Val AP: 0.9048
Epoch 70 | Loss: 1.0751 | Val AUC: 0.8794 | Val AP: 0.8811
Epoch 80 | Loss: 1.0632 | Val AUC: 0.8899 | Val AP: 0.8906
Epoch 90 | Loss: 1.0622 | Val AUC: 0.8965 | Val AP: 0.8953
Epoch 100 | Loss: 1.0493 | Val AUC: 0.8932 | Val AP: 0.8930
Test AUC: 0.8932 | Test AP 0.8930


In [36]:
z = model.encode(test_data.x, test_data.edge_index) 
Ahat = torch.sigmoid(z @ z.T)
Ahat

tensor([[0.5314, 0.5059, 0.4966,  ..., 0.5013, 0.4787, 0.5042],
        [0.5059, 0.6615, 0.5817,  ..., 0.5772, 0.5542, 0.5781],
        [0.4966, 0.5817, 0.5432,  ..., 0.5382, 0.5331, 0.5391],
        ...,
        [0.5013, 0.5772, 0.5382,  ..., 0.5366, 0.5262, 0.5366],
        [0.4787, 0.5542, 0.5331,  ..., 0.5262, 0.5373, 0.5255],
        [0.5042, 0.5781, 0.5391,  ..., 0.5366, 0.5255, 0.5376]],
       grad_fn=<SigmoidBackward0>)

SEAL

In [37]:
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
from scipy.sparse.csgraph import shortest_path

import torch.nn.functional as F
from torch.nn import Conv1d, MaxPool1d, Linear, Dropout, BCEWithLogitsLoss

from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, aggr
from torch_geometric.utils import k_hop_subgraph, to_scipy_sparse_matrix

In [59]:

# Load PubMed dataset
transform = RandomLinkSplit(num_val=0.2, num_test=0.2,  is_undirected=True, split_labels=True)
dataset = Planetoid('.', name='PubMed', transform=transform)


train_data, val_data, test_data = dataset[0]
#print(f"Data set[0]:\n {dataset[0]}")
print(f"Train data: \n{train_data}")
print(f"Val data: \n{val_data}")
print(f"Test data: \n{test_data}")

Train data: 
Data(x=[19717, 500], edge_index=[2, 53192], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], pos_edge_label=[26596], pos_edge_label_index=[2, 26596], neg_edge_label=[26596], neg_edge_label_index=[2, 26596])
Val data: 
Data(x=[19717, 500], edge_index=[2, 53192], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], pos_edge_label=[8864], pos_edge_label_index=[2, 8864], neg_edge_label=[8864], neg_edge_label_index=[2, 8864])
Test data: 
Data(x=[19717, 500], edge_index=[2, 70920], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], pos_edge_label=[8864], pos_edge_label_index=[2, 8864], neg_edge_label=[8864], neg_edge_label_index=[2, 8864])


In [60]:
def seal_processing(dataset, edge_label_index, y):
    data_list = []

    for src, dst in edge_label_index.t().tolist():
        sub_nodes, sub_edge_index, mapping, _ = k_hop_subgraph([src, dst], 2, dataset.edge_index, relabel_nodes=True)
        src, dst = mapping.tolist()

        # Remove target link from the subgraph
        mask1 = (sub_edge_index[0] != src) | (sub_edge_index[1] != dst)
        mask2 = (sub_edge_index[0] != dst) | (sub_edge_index[1] != src)
        sub_edge_index = sub_edge_index[:, mask1 & mask2]

        # Double-radius node labeling (DRNL)
        src, dst = (dst, src) if src > dst else (src, dst)
        adj = to_scipy_sparse_matrix(sub_edge_index, num_nodes=sub_nodes.size(0)).tocsr()

        idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
        adj_wo_src = adj[idx, :][:, idx]

        idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
        adj_wo_dst = adj[idx, :][:, idx]

        # Calculate the distance between every node and the source target node
        d_src = shortest_path(adj_wo_dst, directed=False, unweighted=True, indices=src)
        d_src = np.insert(d_src, dst, 0, axis=0)
        d_src = torch.from_numpy(d_src)

        # Calculate the distance between every node and the destination target node
        d_dst = shortest_path(adj_wo_src, directed=False, unweighted=True, indices=dst-1)
        d_dst = np.insert(d_dst, src, 0, axis=0)
        d_dst = torch.from_numpy(d_dst)

        # Calculate the label z for each node
        dist = d_src + d_dst
        z = 1 + torch.min(d_src, d_dst) + dist // 2 * (dist // 2 + dist % 2 - 1)
        z[src], z[dst], z[torch.isnan(z)] = 1., 1., 0.
        z = z.to(torch.long)

        # Concatenate node features and one-hot encoded node labels (with a fixed number of classes)
        node_labels = F.one_hot(z, num_classes=200).to(torch.float)
        node_emb = dataset.x[sub_nodes]
        node_x = torch.cat([node_emb, node_labels], dim=1)

        # Create data object
        data = Data(x=node_x, z=z, edge_index=sub_edge_index, y=y)
        data_list.append(data)

    return data_list

In [61]:
# Enclosing subgraphs extraction
train_pos_data_list = seal_processing(train_data, train_data.pos_edge_label_index, 1)


In [41]:
train_neg_data_list = seal_processing(train_data, train_data.neg_edge_label_index, 0)

IndexError: index 19716 is out of bounds for dimension 0 with size 19712

In [ ]:

val_pos_data_list = seal_processing(val_data, val_data.pos_edge_label_index, 1)

IndexError: index 18962 is out of bounds for dimension 0 with size 15643

In [ ]:
val_neg_data_list = seal_processing(val_data, val_data.neg_edge_label_index, 0)

In [ ]:
test_pos_data_list = seal_processing(test_data, test_data.pos_edge_label_index, 1)

In [ ]:
test_neg_data_list = seal_processing(test_data, test_data.neg_edge_label_index, 0)

In [ ]:
train_dataset = train_pos_data_list + train_neg_data_list
val_dataset = val_pos_data_list + val_neg_data_list
test_dataset = test_pos_data_list + test_neg_data_list

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
class DGCNN(torch.nn.Module):
    def __init__(self, dim_in, k=30):
        super().__init__()

        # GCN layers
        self.gcn1 = GCNConv(dim_in, 32)
        self.gcn2 = GCNConv(32, 32)
        self.gcn3 = GCNConv(32, 32)
        self.gcn4 = GCNConv(32, 1)

        # Global sort pooling
        self.global_pool = aggr.SortAggregation(k=k)

        # Convolutional layers
        self.conv1 = Conv1d(1, 16, 97, 97)
        self.conv2 = Conv1d(16, 32, 5, 1)
        self.maxpool = MaxPool1d(2, 2)

        # Dense layers
        self.linear1 = Linear(352, 128)
        self.dropout = Dropout(0.5)
        self.linear2 = Linear(128, 1)

    def forward(self, x, edge_index, batch):
        # 1. Graph Convolutional Layers
        h1 = self.gcn1(x, edge_index).tanh()
        h2 = self.gcn2(h1, edge_index).tanh()
        h3 = self.gcn3(h2, edge_index).tanh()
        h4 = self.gcn4(h3, edge_index).tanh()
        h = torch.cat([h1, h2, h3, h4], dim=-1)

        # 2. Global sort pooling
        h = self.global_pool(h, batch)

        # 3. Traditional convolutional and dense layers
        h = h.view(h.size(0), 1, h.size(-1))
        h = self.conv1(h).relu()
        h = self.maxpool(h)
        h = self.conv2(h).relu()
        h = h.view(h.size(0), -1)
        h = self.linear1(h).relu()
        h = self.dropout(h)
        h = self.linear2(h).sigmoid()

        return h

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCNN(train_dataset[0].num_features).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = BCEWithLogitsLoss()

def train():
    model.train()
    total_loss = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out.view(-1), data.y.to(torch.float))
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs

    return total_loss / len(train_dataset)

@torch.no_grad()
def test(loader):
    model.eval()
    y_pred, y_true = [], []

    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        y_pred.append(out.view(-1).cpu())
        y_true.append(data.y.view(-1).cpu().to(torch.float))

    auc = roc_auc_score(torch.cat(y_true), torch.cat(y_pred))
    ap = average_precision_score(torch.cat(y_true), torch.cat(y_pred))

    return auc, ap

In [ ]:

for epoch in range(31):
    loss = train()
    val_auc, val_ap = test(val_loader)
    print(f'Epoch {epoch:>2} | Loss: {loss:.4f} | Val AUC: {val_auc:.4f} | Val AP: {val_ap:.4f}')

test_auc, test_ap = test(test_loader)
print(f'Test AUC: {test_auc:.4f} | Test AP {test_ap:.4f}')

Epoch  0 | Loss: 0.7020 | Val AUC: 0.7103 | Val AP: 0.7293
Epoch  1 | Loss: 0.6404 | Val AUC: 0.7782 | Val AP: 0.8128
Epoch  2 | Loss: 0.6120 | Val AUC: 0.7767 | Val AP: 0.8128
Epoch  3 | Loss: 0.6073 | Val AUC: 0.7773 | Val AP: 0.8161
Epoch  4 | Loss: 0.6046 | Val AUC: 0.7773 | Val AP: 0.8175
Epoch  5 | Loss: 0.6033 | Val AUC: 0.7827 | Val AP: 0.8206
Epoch  6 | Loss: 0.6015 | Val AUC: 0.7829 | Val AP: 0.8231
Epoch  7 | Loss: 0.5997 | Val AUC: 0.7846 | Val AP: 0.8286
Epoch  8 | Loss: 0.5971 | Val AUC: 0.7840 | Val AP: 0.8227
Epoch  9 | Loss: 0.5961 | Val AUC: 0.7800 | Val AP: 0.8256
Epoch 10 | Loss: 0.5949 | Val AUC: 0.7808 | Val AP: 0.8111
Epoch 11 | Loss: 0.5933 | Val AUC: 0.7794 | Val AP: 0.8041
Epoch 12 | Loss: 0.5930 | Val AUC: 0.7808 | Val AP: 0.8019
Epoch 13 | Loss: 0.5906 | Val AUC: 0.7810 | Val AP: 0.8036
Epoch 14 | Loss: 0.5911 | Val AUC: 0.7768 | Val AP: 0.8064
Epoch 15 | Loss: 0.5898 | Val AUC: 0.7769 | Val AP: 0.8008
Epoch 16 | Loss: 0.5885 | Val AUC: 0.7789 | Val AP: 0.80